Caricare il csv come dataframe e mettere ad 1 le i valori della colonna "Anomalous" delle giornate riportate nella tabella del documento docx in allegato.

Apro il csv dei dati e con questo creo un dataframe

In [5]:
import csv
import pandas as pd

csv_path = 'timeSeries2015HotspotD.csv'
with open(csv_path) as csv_file:
  csv_reader = csv.reader(csv_file, delimiter=',')

  df = pd.read_csv(csv_path)

Apro il file Excel con le tabelle delle anomalie suddivise per Cluster e con questo creo 3 dataframe separati.

In [7]:
excel_path = 'doc-convertiti.xlsx'

cluster0 = pd.read_excel(excel_path, sheet_name='Cluster0', usecols=['Data', 'Cluster'])
cluster1 = pd.read_excel(excel_path, sheet_name='Cluster1', usecols=['Data', 'Cluster'])
cluster2 = pd.read_excel(excel_path, sheet_name='Cluster2', usecols=['Data', 'Cluster'])

 cluster0["Data"] = pd.to_datetime(cluster0["Data"])
  cluster1["Data"] = pd.to_datetime(cluster1["Data"])
  cluster2["Data"] = pd.to_datetime(cluster2["Data"])

Itero i dataframe per settare il valore di Anomalous e salvo il relativo .csv

In [9]:
for i, j in df.iterrows():
    for k in cluster0["Data"]:
        if k.day == j['Day'] and k.month == j['Month'] and j['Cluster'] == 0:
            df.at[i, 'Anomalous'] = 1

    for k in cluster1["Data"]:
        if k.day == j['Day'] and k.month == j['Month'] and j['Cluster'] == 1:               
            df.at[i, 'Anomalous'] = 1

    for k in cluster2["Data"]:
        if k.day == j['Day'] and k.month == j['Month'] and j['Cluster'] == 2:
            df.at[i, 'Anomalous'] = 1

df.to_csv('out1.csv')

Eliminare dal dataframe (risultante dal punto 1) tutte le righe con Anomalous a 1

In [11]:
out1_path = 'out1.csv'
with open(out1_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    df = pd.read_csv(out1_path)

    for i, j in df.iterrows():
        if df.at[i, 'Anomalous'] == 1:
            df.drop(i, inplace=True)

    df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)

    df.to_csv('out2.csv', index=False)

Apro il csv salvato in precedenza e creo dataframe separati per la X e la Y della classificazione.
Creo successivamente un oggetto Random Forest per eseguire la classificazione dei cluster e ne calcolo la precisione.

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

out2_path = 'out2.csv'
with open(out2_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    df = pd.read_csv(out2_path)

    target_cols = ['Cluster']
    data_cols = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'h8', 'h9', 'h10', 'h11', 'h12', 'h13', 'h14',
                'h15', 'h16', 'h17', 'h18', 'h19', 'h20', 'h21', 'h22', 'h23']

    X = df[data_cols]
    Y = df[target_cols]

    X, y = make_classification(n_samples=328)

    clf = RandomForestClassifier(max_depth=2, random_state=0)

    clf.fit(X, y)

    Y_pred = clf.predict(X)

    print("Precisione:", clf.score(X, Y, sample_weight=None))

Precisione: 0.4634146341463415
